# Immoweb data analysis 

### Importing libraries 

In [99]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score



###  Load the Data

In [100]:
data_path = 'data/data_20240313_cleaned.csv'
df_cleaned = pd.read_csv(data_path)

### Initial Data Cleaning with Ordinal and One-Hot Encoding

In [101]:
# Define the function to calculate IQR bounds
def calculate_bounds(data, feature):
    Q1 = data[feature].quantile(0.25)
    Q3 = data[feature].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return lower_bound, upper_bound, Q1, Q3

# Create a dictionary of features to check for outliers
features_to_check = ['price', 'surface_land_sqm', 'total_area_sqm', 'nbr_bedrooms']

# Initialize a dictionary to hold outlier ranges for each feature
outlier_ranges = {}

# Calculate and print the number of outliers for each feature
for feature in features_to_check:
    lower_bound, upper_bound, Q1, Q3 = calculate_bounds(df_cleaned, feature)
    outliers = df_cleaned[(df_cleaned[feature] < lower_bound) | (df_cleaned[feature] > upper_bound)]
    outlier_ranges[feature] = {'lower_bound': lower_bound, 'upper_bound': upper_bound, 'Q1': Q1, 'Q3': Q3}
    print(f"Outliers for {feature}: {len(outliers)}")
    print(f"Range of outliers for {feature}: [{lower_bound}, {upper_bound}]")
    print(f"Interquartile range (IQR) for {feature}: [{Q1}, {Q3}]")
    print()

# Filter out the outliers
filtered_data = df_cleaned[
    (df_cleaned['price'] >= outlier_ranges['price']['lower_bound']) & (df_cleaned['price'] <= outlier_ranges['price']['upper_bound']) &
    (df_cleaned['surface_land_sqm'] >= outlier_ranges['surface_land_sqm']['lower_bound']) & (df_cleaned['surface_land_sqm'] <= outlier_ranges['surface_land_sqm']['upper_bound']) &
    (df_cleaned['total_area_sqm'] >= outlier_ranges['total_area_sqm']['lower_bound']) & (df_cleaned['total_area_sqm'] <= outlier_ranges['total_area_sqm']['upper_bound']) &
    (df_cleaned['nbr_bedrooms'] >= outlier_ranges['nbr_bedrooms']['lower_bound']) & (df_cleaned['nbr_bedrooms'] <= outlier_ranges['nbr_bedrooms']['upper_bound'])
].copy()


# Proceed to apply the mapping without warnings
filtered_data['epc_encoded'] = filtered_data['epc'].map(epc_mapping)

# Ordinal encoding for EPC
epc_mapping = {
    'A++': 1, 'A+': 2, 'A': 3, 'B': 4, 'C': 5, 'D': 6, 'E': 7, 'F': 8, 'G': 9, 'MISSING': -1
}
filtered_data['epc_encoded'] = filtered_data['epc'].map(epc_mapping)

# One-hot encoding for categorical variables
categorical_columns = ['property_type', 'heating_type', 'state_building', 'subproperty_type', 
                       'region', 'province', 'locality', 'equipped_kitchen']  
df_encoded = pd.get_dummies(filtered_data, columns=categorical_columns)

# After one-hot encoding, handle NaN values for all columns
columns_to_fill = [col for col in df_encoded.columns if col not in ('price', 'id', 'epc')]
df_encoded[columns_to_fill] = df_encoded[columns_to_fill].fillna(-1)


Outliers for price: 5800
Range of outliers for price: [-77500.0, 774500.0]
Interquartile range (IQR) for price: [242000.0, 455000.0]

Outliers for surface_land_sqm: 3534
Range of outliers for surface_land_sqm: [-832.5, 1787.5]
Interquartile range (IQR) for surface_land_sqm: [150.0, 805.0]

Outliers for total_area_sqm: 3995
Range of outliers for total_area_sqm: [-50.0, 326.0]
Interquartile range (IQR) for total_area_sqm: [91.0, 185.0]

Outliers for nbr_bedrooms: 8376
Range of outliers for nbr_bedrooms: [0.5, 4.5]
Interquartile range (IQR) for nbr_bedrooms: [2.0, 3.0]



### Handle Missing Values for Numeric Columns

In [102]:
numeric_columns = ['construction_year', 'total_area_sqm', 'surface_land_sqm', 'nbr_frontages', 
                   'fl_furnished', 'fl_open_fire', 'fl_terrace', 'terrace_sqm', 'fl_garden', 
                   'garden_sqm', 'fl_swimming_pool', 'fl_floodzone', 'primary_energy_consumption_sqm', 
                   'fl_double_glazing', 'cadastral_income', 'latitude', 'longitude']
df_encoded[numeric_columns] = df_encoded[numeric_columns].fillna(-1)


### Data Preparation for Modeling

In [103]:
X = df_encoded.drop(columns=['price', 'id', 'epc'])
y = df_encoded['price']


### Split the Data

In [104]:
# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


### Define Preprocessing and Modeling Pipeline

In [105]:
# Preprocessing for numeric features: scaling
numeric_features = X.select_dtypes(include=['int64', 'float64', 'uint8']).columns
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[('scaler', StandardScaler())]), numeric_features)
    ],
    remainder='passthrough'  # Include this to keep one-hot encoded columns without scaling
)

# Model pipeline
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])


### Train the Model

In [106]:
model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  Index(['zip_code', 'latitude', 'longitude', 'construction_year',
       'total_area_sqm', 'surface_land_sqm', 'nbr_frontages', 'nbr_bedrooms',
       'fl_furnished', 'fl_open_fire', 'fl_terrace', 'terrace_sqm',
       'fl_garden', 'garden_sqm', 'fl_swimming_pool', 'fl_floodzone',
       'primary_energy_consumption_sqm', 'fl_double_glazing',
       'cadastral_income', 'epc_encoded'],
      dtype='object'))])),
                ('regressor', LinearRegression())])

### Evaluate the Model

In [107]:
# Predict on test set and evaluate
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error (MSE): {mse}')
print(f'Root Mean Squared Error (RMSE): {rmse}')
print(f'Mean Absolute Error (MAE): {mae}')
print(f'R-squared (R2): {r2}')



Mean Squared Error (MSE): 6596398433.08112
Root Mean Squared Error (RMSE): 81218.21490947163
Mean Absolute Error (MAE): 59725.81181038006
R-squared (R2): 0.6762459645240256
